In [1]:
import time
from pyspark.sql.functions import window
from pyspark.sql import functions as f

In [2]:
import numpy as np
from IPython.display import clear_output

In [3]:
# from pyspark import SparkContext
# from pyspark.streaming import StreamingContext
# sc = SparkContext("local", "WordCount")
# ssc = StreamingContext(sc, 10) // Stream data every 10 sec
# lines = ssc.textFileStream("/home/karthiamrita/textdata")
# words = lines.flatMap(lambda line: line.split(" "))
# # Count each word in each batch
# pairs = words.map(lambda word: (word, 1))
# wordCounts = pairs.reduceByKey(lambda x, y: x + y)
# # Print the first 10 
# elements of each RDD generated in this DStream to the console
# wordCounts.pprint()
# ssc.start() # Start the computation
# ssc.awaitTermination()

In [4]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import Row, SparkSession
from pyspark.sql.types import StructType
import pyspark.sql.types as tp
# Create a spark Session
spark = SparkSession.builder.appName("StructuredStreaming").getOrCreate()


In [5]:
user_schema = tp.StructType([
  tp.StructField(name= 'mobileno',dataType= tp.StringType(),  nullable= True),
  tp.StructField(name= 'value',dataType= tp.StringType(),   nullable= True)
])

In [6]:

# create the schema for dataframe
#userSchema = StructType().add("mobileno", StringType()).add("dateandtime", float).add("acc",float).add("gyro",float).add("weight",Integer).add("height",Integer).add("Gender",string).add("age",Integer)
# read the data from folder - contains all the CSV file
csvDF = spark.readStream.format("csv").option("header", True).schema(user_schema).load("/home/jovyan/work/mldb_proj//data")
csvDF.isStreaming
#csvDF.show()
#query = csvDF.writeStream.outputMode("append").format("console").start()
#query.awaitTermination()
#query=csvDF.writeStream.format("csv").trigger(processingTime="10 seconds").option("checkpointLocation", "/home/jovyan/work/mldb_proj").option("path","/home/jovyan/work/mldb_proj//file").outputMode("append").start()


True

In [7]:
csvDF.createOrReplaceTempView("mobile_position");
#orders_find_text = spark.sql("SELECT * FROM orders_find")
sensor_write_stream = csvDF \
        .writeStream \
        .trigger(processingTime='5 seconds') \
        .outputMode("append") \
        .option("truncate", "false") \
        .format('memory') \
        .queryName("sensorTable") \
        .start()

In [10]:
while True:
    count=spark.sql("SELECT count(*) FROM sensorTable" )
    count.show()
   # df = spark.sql("SELECT * FROM sensorTable" )
#    # acc1, gyro1  = get_data_mobile1(df.tail(2))
#     print(acc1, gyro1)
    time.sleep(10)
    clear_output(wait=True)

+--------+
|count(1)|
+--------+
|      40|
+--------+



KeyboardInterrupt: 

In [54]:
df

DataFrame[mobileno: string, value: string]

In [35]:
df.shape()

AttributeError: 'DataFrame' object has no attribute 'shape'

In [29]:
acc1, gyro1  = get_data_mobile1(df.tail(2))
print(acc1, gyro1)

9.91517029606653 0.0


In [25]:
def get_data_mobile1(data):
    
    mobile1_values = data[0][1]
    x = list(map(float, mobile1_values.split('_')))
    mobile = "mobile1"
    sqrt_acceler  = 0
    sqrt_gyro = 0
    i = 0
    for item in x:  
        i = i+1
        if item == 3.0:
            acceleros=[x[i],x[i+1],x[i+2]]
            sqrt_acceler = np.sqrt(sum(j*j for j in acceleros))
#             print(sqrt_acceler)

        if item == 4.0:
            gyros=[x[i],x[i+1],x[i+2]]
            sqrt_gyro = np.sqrt(sum(j*j for j in gyros))
#             print(sqrt_gyro)
            
    return float(sqrt_acceler), float(sqrt_gyro)
   # return mobile, mobile1_values

In [26]:
def get_data_mobile2(data):
    
    mobile1_values = data[1][1]
    x = list(map(float, mobile1_values.split('_')))
    mobile = "mobile1"
    sqrt_acceler  = 0
    sqrt_gyro = 0
    i = 0
    for item in x:  
        i = i+1
        if item == 3.0:
            acceleros=[x[i],x[i+1],x[i+2]]
            sqrt_acceler = np.sqrt(sum(j*j for j in acceleros))
#             print(sqrt_acceler)

        if item == 4.0:
            gyros=[x[i],x[i+1],x[i+2]]
            sqrt_gyro = np.sqrt(sum(j*j for j in gyros))
#             print(sqrt_gyro)
            
    return mobile, float(sqrt_acceler), float(sqrt_gyro)
   # return mobile, mobile1_values

In [57]:
# df = spark.sql("SELECT * FROM sensorTable \
#                 CLUSTER BY mobileno" )
# #a=df.groupby("mobileno")
# df.show()

+--------+--------+----+
|mobileno|     acc|gyro|
+--------+--------+----+
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
| mobile1|9.915171| 0.0|
+--------+--------+----+
only showing top 20 rows



In [60]:
df.tail(2)

[Row(mobileno='mobile1', acc=9.915170669555664, gyro=0.0),
 Row(mobileno='mobile2', acc=9.841073036193848, gyro=0.0)]

In [51]:
size = df.count()
df.rdd.zipWithIndex()\
  .filter(lambda x : x[1] == 0 or x[1] == size-1)\
  .map(lambda x : x[0].acc)\
  .collect()

[9.948163032531738, 9.841073036193848]

In [16]:
# # query=csvDF.writeStream.format("csv").option("checkpointLocation", "/home/jovyan/work/mldb_proj").option("path","/home/jovyan/work/mldb_proj//file").outputMode("append").toTable("myTable").start()
# spark.read.table("myTable").show()

In [86]:
# def process_row(row):
    
#     # Write row to storage
#     print(row)
# csvDF = spark.readStream.format("csv").option("header", True).schema(user_schema).load("/home/jovyan/work/mldb_proj//data")
# query = csvDF.writeStream.foreach(process_row).start()  

In [67]:
# df = spark.readStream \
#     .format("rate") \
#     .option("rowsPerSecond", 10) \
#     .load()

In [ ]:
# # Write the streaming DataFrame to a table
# df.writeStream \
#     .option("checkpointLocation", "path/to/checkpoint/dir") \
#     .toTable("myTable")
# # Check the table result
# spark.read.table("myTable").show()

In [68]:
# df.show()
# query.stop()

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
rate

In [23]:
csvDF.printSchema()

root
 |-- mobileno: string (nullable = true)
 |-- dateandtime: string (nullable = true)
 |-- acc: float (nullable = true)
 |-- gyro: float (nullable = true)
 |-- weight: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: integer (nullable = true)



In [24]:
for x in csvDF:
    print(x)

Column<'mobileno'>
Column<'dateandtime'>
Column<'acc'>
Column<'gyro'>
Column<'weight'>
Column<'height'>
Column<'gender'>
Column<'age'>


In [22]:
#csvDF = spark.readStream.format("csv").option("header", True).schema(user_schema).csv("/home/jovyan/work/mldb_proj//data")
csvDF = spark.readStream.option("sep", ",").option("header" , True).schema(user_schema).csv("/home/jovyan/work/mldb_proj//data")
#csvDF_time = csvDF.withColumn("time_st",f.current_timestamp())
windowed_df = csvDF.agg(f.sum("acc1"))
query = ( windowed_df.writeStream.outputMode("complete").format("console").queryName("counts").start() )
query.awaitTermination()


AnalysisException: cannot resolve '`acc1`' given input columns: [acc, age, dateandtime, gender, gyro, height, mobileno, weight];
'Aggregate [sum('acc1) AS sum(acc1)#195]
+- StreamingRelation DataSource(org.apache.spark.sql.SparkSession@4c545b73,csv,List(),Some(StructType(StructField(mobileno,StringType,true), StructField(dateandtime,StringType,true), StructField(acc,FloatType,true), StructField(gyro,FloatType,true), StructField(weight,IntegerType,true), StructField(height,IntegerType,true), StructField(gender,StringType,true), StructField(age,IntegerType,true))),List(),None,Map(sep -> ,, header -> true, path -> /home/jovyan/work/mldb_proj//data),None), FileSource[/home/jovyan/work/mldb_proj//data], [mobileno#170, dateandtime#171, acc#172, gyro#173, weight#174, height#175, gender#176, age#177]


In [1]:
query.stop()

NameError: name 'query' is not defined

In [38]:
# display(csvDF)

In [39]:
# csvDF.printSchema()